## استكشاف التجميع باستخدام K-Means في R ومبادئ البيانات المنظمة

### [**اختبار ما قبل المحاضرة**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

في هذا الدرس، ستتعلم كيفية إنشاء مجموعات باستخدام حزمة Tidymodels وحزم أخرى في نظام R البيئي (سنطلق عليها الأصدقاء 🧑‍🤝‍🧑)، بالإضافة إلى مجموعة بيانات الموسيقى النيجيرية التي قمت باستيرادها سابقًا. سنغطي أساسيات K-Means للتجميع. تذكر أنه، كما تعلمت في الدرس السابق، هناك العديد من الطرق للعمل مع المجموعات والطريقة التي تستخدمها تعتمد على بياناتك. سنجرب K-Means لأنها التقنية الأكثر شيوعًا للتجميع. لنبدأ!

المصطلحات التي ستتعرف عليها:

-   تقييم Silhouette

-   طريقة Elbow

-   القصور الذاتي (Inertia)

-   التباين (Variance)

### **المقدمة**

[التجميع باستخدام K-Means](https://wikipedia.org/wiki/K-means_clustering) هو طريقة مشتقة من مجال معالجة الإشارات. تُستخدم لتقسيم وتصنيف مجموعات البيانات إلى `k مجموعات` بناءً على التشابه في خصائصها.

يمكن تصور المجموعات كـ [مخططات فورونوي](https://wikipedia.org/wiki/Voronoi_diagram)، التي تتضمن نقطة (أو "بذرة") ومنطقتها المقابلة.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>رسم توضيحي بواسطة Jen Looper</figcaption>

خطوات التجميع باستخدام K-Means هي كالتالي:

1. يبدأ عالم البيانات بتحديد عدد المجموعات المراد إنشاؤها.

2. بعد ذلك، يختار الخوارزم عشوائيًا K ملاحظات من مجموعة البيانات لتكون بمثابة المراكز الأولية للمجموعات (أي المراكز).

3. يتم بعد ذلك تعيين كل ملاحظة متبقية إلى أقرب مركز.

4. يتم حساب المتوسط الجديد لكل مجموعة ويتم نقل المركز إلى هذا المتوسط.

5. الآن وبعد إعادة حساب المراكز، يتم التحقق مرة أخرى من كل ملاحظة لمعرفة ما إذا كانت قد تكون أقرب إلى مجموعة مختلفة. يتم إعادة تعيين جميع العناصر باستخدام متوسطات المجموعات المحدثة. يتم تكرار خطوات تعيين المجموعات وتحديث المراكز بشكل تكراري حتى تتوقف تعيينات المجموعات عن التغيير (أي عند تحقيق التقارب). عادةً ما تنتهي الخوارزمية عندما ينتج عن كل تكرار جديد حركة ضئيلة للمراكز وتصبح المجموعات ثابتة.

<div>

> لاحظ أنه بسبب العشوائية في اختيار الملاحظات الأولية k المستخدمة كمراكز بداية، قد نحصل على نتائج مختلفة قليلاً في كل مرة نطبق فيها الإجراء. لهذا السبب، تستخدم معظم الخوارزميات عدة *بدايات عشوائية* وتختار التكرار الذي يحتوي على أقل WCSS. لذلك، يُوصى بشدة دائمًا بتشغيل K-Means مع عدة قيم لـ *nstart* لتجنب *الوقوع في الحد الأدنى المحلي غير المرغوب فيه.*

</div>

توضح هذه الرسوم المتحركة القصيرة باستخدام [الأعمال الفنية](https://github.com/allisonhorst/stats-illustrations) لـ Allison Horst عملية التجميع:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>عمل فني بواسطة @allison_horst</figcaption>

سؤال أساسي يطرح نفسه في التجميع: كيف تعرف عدد المجموعات التي يجب تقسيم بياناتك إليها؟ أحد عيوب استخدام K-Means هو أنك ستحتاج إلى تحديد `k`، أي عدد `المراكز`. لحسن الحظ، تساعد طريقة `elbow` في تقدير قيمة بداية جيدة لـ `k`. ستجربها بعد قليل.

### 

**المتطلبات الأساسية**

سنبدأ من حيث توقفنا في [الدرس السابق](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb)، حيث قمنا بتحليل مجموعة البيانات، وإنشاء العديد من التصورات، وتصفيتها إلى الملاحظات ذات الأهمية. تأكد من الاطلاع عليه!

سنحتاج إلى بعض الحزم لإكمال هذا الدرس. يمكنك تثبيتها باستخدام: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

بدلاً من ذلك، يقوم النص أدناه بالتحقق مما إذا كانت الحزم المطلوبة لإكمال هذا الدرس مثبتة لديك، ويقوم بتثبيتها إذا كانت مفقودة.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


لننطلق بسرعة!

## 1. رقصة مع البيانات: حصر الأنواع الموسيقية في الثلاثة الأكثر شهرة

هذا تذكير بما قمنا به في الدرس السابق. لنقم بتقطيع وتحليل بعض البيانات!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 هذا سار بشكل جيد!

## 2. المزيد من استكشاف البيانات.

ما مدى نظافة هذه البيانات؟ دعونا نتحقق من القيم الشاذة باستخدام مخططات الصندوق. سنركز على الأعمدة الرقمية التي تحتوي على عدد أقل من القيم الشاذة (على الرغم من أنه يمكنك تنظيف القيم الشاذة). يمكن لمخططات الصندوق أن تعرض نطاق البيانات وستساعد في اختيار الأعمدة التي يمكن استخدامها. لاحظ أن مخططات الصندوق لا تعرض التباين، وهو عنصر مهم للحصول على بيانات قابلة للتجميع بشكل جيد. يرجى الاطلاع على [هذا النقاش](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) لمزيد من القراءة.

تُستخدم [مخططات الصندوق](https://en.wikipedia.org/wiki/Box_plot) لتمثيل توزيع البيانات الرقمية بشكل رسومي، لذا دعونا نبدأ بـ *اختيار* جميع الأعمدة الرقمية إلى جانب الأنواع الموسيقية الشهيرة.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


شاهد كيف تجعل أداة التحديد `where` هذا الأمر سهلاً 💁؟ استكشف وظائف أخرى مشابهة [هنا](https://tidyselect.r-lib.org/).

بما أننا سنقوم بإنشاء مخطط صندوقي لكل خاصية رقمية ونرغب في تجنب استخدام الحلقات، دعونا نعيد تنسيق بياناتنا إلى صيغة *أطول* تتيح لنا الاستفادة من `facets` - وهي رسوم فرعية تعرض كل منها مجموعة فرعية من البيانات.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


أطول بكثير! حان الوقت الآن لبعض `ggplots`! إذاً، ما هو `geom` الذي سنستخدمه؟


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


!سهل جدًا

الآن يمكننا أن نلاحظ أن هذه البيانات تحتوي على بعض الضوضاء: من خلال مراقبة كل عمود كـ boxplot، يمكنك رؤية القيم الشاذة. يمكنك مراجعة مجموعة البيانات وإزالة هذه القيم الشاذة، ولكن ذلك سيجعل البيانات محدودة جدًا.

في الوقت الحالي، دعونا نختار الأعمدة التي سنستخدمها في تمرين التجميع الخاص بنا. لنختار الأعمدة الرقمية ذات النطاقات المتشابهة. يمكننا ترميز `artist_top_genre` كرقم، ولكن سنقوم بتجاهله في الوقت الحالي.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. حساب التجميع باستخدام k-means في R

يمكننا حساب k-means في R باستخدام الدالة المدمجة `kmeans`. راجع `help("kmeans()")`. تقبل دالة `kmeans()` إطار بيانات يحتوي على أعمدة رقمية فقط كوسيطة رئيسية.

الخطوة الأولى عند استخدام التجميع باستخدام k-means هي تحديد عدد المجموعات (k) التي سيتم إنشاؤها في الحل النهائي. نحن نعلم أن هناك 3 أنواع من الأغاني التي استخرجناها من مجموعة البيانات، لذا دعونا نجرب 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


يحتوي كائن kmeans على عدة معلومات موضحة بشكل جيد في `help("kmeans()")`. في الوقت الحالي، دعونا نركز على بعض النقاط. نلاحظ أن البيانات قد تم تقسيمها إلى 3 مجموعات بأحجام 65، 110، 111. كما يحتوي الناتج على مراكز المجموعات (المتوسطات) للمجموعات الثلاث عبر المتغيرات الخمسة.

يمثل متجه التجميع تعيين المجموعة لكل ملاحظة. دعونا نستخدم وظيفة `augment` لإضافة تعيين المجموعة إلى مجموعة البيانات الأصلية.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


لقد قمنا للتو بتقسيم مجموعة البيانات الخاصة بنا إلى مجموعة من 3 مجموعات. إذًا، ما مدى جودة التجميع الذي قمنا به 🤷؟ دعونا نلقي نظرة على `Silhouette score`.

### **درجة Silhouette**

[تحليل Silhouette](https://en.wikipedia.org/wiki/Silhouette_(clustering)) يمكن استخدامه لدراسة المسافة الفاصلة بين المجموعات الناتجة. تتراوح هذه الدرجة بين -1 و 1، وإذا كانت الدرجة قريبة من 1، فإن المجموعة تكون كثيفة ومفصولة بشكل جيد عن المجموعات الأخرى. أما القيمة القريبة من 0 فتشير إلى مجموعات متداخلة مع عينات قريبة جدًا من حدود القرار للمجموعات المجاورة. [المصدر](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

طريقة Silhouette المتوسطة تحسب متوسط Silhouette للملاحظات لقيم مختلفة من *k*. تشير درجة Silhouette المتوسطة العالية إلى تجميع جيد.

وظيفة `silhouette` في حزمة التجميع تُستخدم لحساب عرض Silhouette المتوسط.

> يمكن حساب Silhouette باستخدام أي [مسافة](https://en.wikipedia.org/wiki/Distance "Distance") مثل [المسافة الإقليدية](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") أو [مسافة مانهاتن](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance") التي ناقشناها في [الدرس السابق](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


نتيجتنا هي **.549**، مما يعني أنها تقع في المنتصف. يشير هذا إلى أن بياناتنا ليست مناسبة بشكل خاص لهذا النوع من التجميع. دعونا نرى ما إذا كان بإمكاننا تأكيد هذا الحدس بصريًا. توفر [حزمة factoextra](https://rpkgs.datanovia.com/factoextra/index.html) وظائف (`fviz_cluster()`) لتصور التجميع.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


يشير التداخل في المجموعات إلى أن بياناتنا ليست مناسبة تمامًا لهذا النوع من التجميع، ولكن دعونا نستمر.

## 4. تحديد العدد الأمثل للمجموعات

سؤال أساسي غالبًا ما يطرح في تجميع K-Means هو - بدون وجود تسميات معروفة للفئات، كيف تعرف عدد المجموعات التي يجب تقسيم بياناتك إليها؟

إحدى الطرق التي يمكننا تجربتها لمعرفة ذلك هي استخدام عينة من البيانات لـ `إنشاء سلسلة من نماذج التجميع` مع زيادة تدريجية في عدد المجموعات (على سبيل المثال من 1 إلى 10)، وتقييم مقاييس التجميع مثل **مقياس Silhouette.**

دعونا نحدد العدد الأمثل للمجموعات من خلال حساب خوارزمية التجميع لقيم مختلفة لـ *k* وتقييم **مجموع المربعات داخل المجموعات** (WCSS). يقيس مجموع المربعات داخل المجموعات (WCSS) مدى تماسك التجميع، ونريد أن يكون صغيرًا قدر الإمكان، حيث تشير القيم الأقل إلى أن نقاط البيانات أقرب إلى بعضها البعض.

دعونا نستكشف تأثير الخيارات المختلفة لـ `k`، من 1 إلى 10، على هذا التجميع.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


الآن بعد أن حصلنا على مجموع مربعات المسافات داخل المجموعات (tot.withinss) لكل خوارزمية تجميع مع المركز *k*، نستخدم [طريقة الكوع](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) لتحديد العدد الأمثل للمجموعات. تتضمن الطريقة رسم WCSS كدالة لعدد المجموعات، واختيار [نقطة الكوع في المنحنى](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") كعدد المجموعات التي سيتم استخدامها.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


يُظهر الرسم البياني انخفاضًا كبيرًا في WCSS (مما يعني زيادة *التماسك*) مع زيادة عدد المجموعات من واحدة إلى اثنتين، وانخفاضًا ملحوظًا آخر من اثنتين إلى ثلاث مجموعات. بعد ذلك، يصبح الانخفاض أقل وضوحًا، مما يؤدي إلى ظهور ما يُعرف بـ `elbow` 💪 في الرسم البياني عند حوالي ثلاث مجموعات. هذه إشارة جيدة على وجود مجموعتين إلى ثلاث مجموعات منفصلة بشكل معقول من نقاط البيانات.

يمكننا الآن المضي قدمًا واستخراج نموذج التجميع حيث `k = 3`:

> `pull()`: تُستخدم لاستخراج عمود واحد
>
> `pluck()`: تُستخدم لفهرسة هياكل البيانات مثل القوائم


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


رائع! لنقم الآن بعرض المجموعات التي حصلنا عليها. هل ترغب في إضافة بعض التفاعلية باستخدام `plotly`؟


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


ربما كنا نتوقع أن يكون لكل مجموعة (ممثلة بألوان مختلفة) أنواع مميزة (ممثلة بأشكال مختلفة).

لنلقِ نظرة على دقة النموذج.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


دقة هذا النموذج ليست سيئة، لكنها ليست ممتازة أيضًا. قد يكون السبب أن البيانات لا تناسب بشكل جيد خوارزمية K-Means Clustering. هذه البيانات غير متوازنة بشكل كبير، قليلة الترابط، وهناك تباين كبير بين قيم الأعمدة مما يجعل عملية التجميع صعبة. في الواقع، من المحتمل أن تكون المجموعات التي تتشكل متأثرة بشكل كبير أو منحازة بفعل الفئات الثلاثة التي قمنا بتعريفها أعلاه.

ومع ذلك، كانت هذه تجربة تعليمية رائعة!

في توثيق Scikit-learn، يمكنك أن ترى أن نموذجًا مثل هذا، حيث المجموعات ليست محددة بوضوح، يعاني من مشكلة "التباين":

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>رسم توضيحي من Scikit-learn</figcaption>



## **التباين**

يُعرف التباين بأنه "متوسط الفروق المربعة عن المتوسط" [المصدر](https://www.mathsisfun.com/data/standard-deviation.html). في سياق مشكلة التجميع هذه، يشير إلى البيانات التي تميل أرقامها إلى التباعد بشكل كبير عن المتوسط.

✅ هذه فرصة رائعة للتفكير في جميع الطرق التي يمكنك من خلالها تصحيح هذه المشكلة. هل يمكنك تعديل البيانات أكثر؟ استخدام أعمدة مختلفة؟ تجربة خوارزمية مختلفة؟ تلميح: جرب [تقييس البيانات](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) لتطبيعها واختبار أعمدة أخرى.

> جرب هذا '[حاسبة التباين](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' لفهم المفهوم بشكل أفضل.

------------------------------------------------------------------------

## **🚀التحدي**

اقضِ بعض الوقت مع هذا الدفتر، وعدّل المعلمات. هل يمكنك تحسين دقة النموذج عن طريق تنظيف البيانات أكثر (مثل إزالة القيم الشاذة)؟ يمكنك استخدام الأوزان لإعطاء أهمية أكبر لبعض عينات البيانات. ما الذي يمكنك فعله أيضًا لإنشاء مجموعات أفضل؟

تلميح: جرب تقييس البيانات. هناك كود معلق في الدفتر يضيف التقييس القياسي لجعل أعمدة البيانات أكثر تشابهًا من حيث النطاق. ستلاحظ أنه بينما ينخفض مؤشر السيلويت، فإن "الانحناء" في الرسم البياني للكوع يصبح أكثر سلاسة. هذا لأن ترك البيانات دون تقييس يسمح للبيانات ذات التباين الأقل بأن تحمل وزنًا أكبر. اقرأ المزيد عن هذه المشكلة [هنا](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**اختبار ما بعد المحاضرة**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **المراجعة والدراسة الذاتية**

-   ألقِ نظرة على محاكي K-Means [مثل هذا](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). يمكنك استخدام هذه الأداة لتصور نقاط البيانات النموذجية وتحديد مراكزها. يمكنك تعديل عشوائية البيانات، وعدد المجموعات، وعدد المراكز. هل يساعدك هذا في الحصول على فكرة عن كيفية تجميع البيانات؟

-   ألقِ نظرة أيضًا على [هذا المستند عن K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) من جامعة ستانفورد.

هل تريد تجربة مهاراتك المكتسبة حديثًا في التجميع على مجموعات بيانات تناسب K-Means Clustering؟ يرجى الاطلاع على:

-   [تدريب وتقييم نماذج التجميع](https://rpubs.com/eR_ic/clustering) باستخدام Tidymodels وأدوات أخرى

-   [تحليل التجميع باستخدام K-Means](https://uc-r.github.io/kmeans_clustering)، دليل UC لتحليل البيانات باستخدام R

- [التجميع باستخدام K-Means ومبادئ البيانات المنظمة](https://www.tidymodels.org/learn/statistics/k-means/)

## **التكليف**

[جرب طرق تجميع مختلفة](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## شكر خاص إلى:

[Jen Looper](https://www.twitter.com/jenlooper) لإنشاء النسخة الأصلية من هذا النموذج بلغة Python ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) لابتكار الرسوم التوضيحية الرائعة التي تجعل R أكثر ترحيبًا وجاذبية. يمكنك العثور على المزيد من الرسوم التوضيحية في [معرضها](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

تعلم سعيد،

[Eric](https://twitter.com/ericntay)، سفير Microsoft Learn Student الذهبي.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>عمل فني من @allison_horst</figcaption>



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للحصول على معلومات حساسة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
